In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ringtone/phone-ringtone-telephone-324474.mp3


In [2]:
!pip install -U langchain langchain-core langchain-community langchain-openai \
  faiss-cpu google-cloud-texttospeech pydub reportlab gradio langgraph --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 51.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.1/188.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━

# 🧠 Project Title: "Global MedAssist: Multi-Agent System for Travel Health Insurance Operations"
# 🩺 Domain: Healthcare Operations & Travel Insurance
## 🌍 Real-World Scenario:

Your company provides health coverage for travelers around the world. When clients experience medical issues abroad—ranging from minor outpatient consultations to critical emergency admissions—a coordinated response is needed. Currently, a human operations agent manages the workflow. This project proposes a multi-agent system to automate and streamline that process. Furthermore, adds a LLM in form of a smart phone app that would speed the contact between the aptient and the insurance company in terms of commmunication, referral to Hospitals within our network and sending medical reports and images of the injuries so that our medical team can assess them. 

# Capstone Project: Multi-Agent System for Travel Health Insurance Operations

## Objective:
To build a simulated multi-agent system that replaces a human operations agent in a global travel health insurance company. This system assists clients who encounter medical issues while traveling abroad.


# -----------------------------------
# 🔧 SECTION 1: Agent System Definition
# -----------------------------------

## 1.1 Agent Roles and Responsibilities

In [3]:
from collections import OrderedDict

# 1.1 Agent Roles and Responsibilities
agent_roles = OrderedDict([
    ("ClientInteractionAgent", 
     "First point of contact for the traveler (client). Captures incident details via multilingual chat or voice, "
     "identifies the client and policy using name, ID, or geolocation, and triggers the triage process with a unique case ID. "
     "Tech: NLP, Google Cloud TTS, contextual empathy prompts."),
    
    ("TriageMedicalAssessmentAgent", 
     "Classifies the case based on urgency and symptoms. Evaluates symptom severity using clinical rule sets, "
     "determines care level (outpatient, ER, inpatient), and escalates life-threatening cases to the Repatriation Agent. "
     "Tech: Decision trees, symptom checkers, rule-based protocols."),
    
    ("ProviderNetworkAgent", 
     "Finds suitable nearby medical facilities based on location, language, specialty, and availability. Sends appointment requests, "
     "confirms bookings, and logs provider responses with estimated wait times. "
     "Tech: Fuzzy location matching, mocked RAG for provider directories."),
    
    ("MedicalDocumentationAgent", 
     "Collects, translates, and formats medical documents including discharge summaries, invoices, and diagnostics. Translates reports if necessary, "
     "and extracts key data for policy and decision validation. "
     "Tech: OCR, translation APIs, entity extraction."),
    
    ("PolicyValidationAgent", 
     "Verifies if the requested treatment is covered by the client’s policy. Retrieves policy terms, matches treatment details, "
     "and flags exclusions, co-pays, or missing documentation. "
     "Tech: Knowledge graph queries, mock policy lookup APIs."),
    
    ("RepatriationPlannerAgent", 
     "Plans and coordinates the client’s transport back home. Assesses feasibility (commercial flight, air ambulance, ground transport), "
     "coordinates with local and home providers, and ensures medical escort and fit-to-fly documentation. "
     "Tech: Scenario planning, cost estimation, real-time logistics."),
    
    ("MedicalDecisionAgent", 
     "Supports complex clinical and operational decisions. Offers second opinions, validates provider suitability, "
     "and interfaces with clinical guidelines and expert systems. "
     "Tech: Rule-based reasoning, LLM summarization."),
    
    ("ComplianceConsentAgent", 
     "Handles legal and consent-related tasks. Ensures clients have accepted data sharing terms, generates e-consent forms, "
     "and logs compliance for applicable jurisdictions (e.g., GDPR, HIPAA). "
     "Tech: Template generation, e-signatures, legal compliance logic."),
    
    ("CountryCareLevelAgent", 
     "Determines the risk level of the country (Level 1–3). Triggers escalation to the medical team if the client is admitted in a Level 3 country. "
     "Also handles special cases like ICU admissions or multi-victim incidents."),
    
    ("OrchestratorAgent", 
     "Manages workflow, routes tasks between agents, and monitors overall progress. Handles exceptions, timeouts, "
     "and logs KPIs for comparison with human workflows. "
     "Tech: LangGraph orchestration, event logging, retry policies.")
])

# Optional: Validate agent roles
assert "ClientInteractionAgent" in agent_roles
assert len(agent_roles) == 10

# Print out each agent and their role with numbering
for idx, (agent, task) in enumerate(agent_roles.items(), 1):
    print(f"\n{idx}. 🤖 {agent}:")
    print(f"   ➔ {task}")





1. 🤖 ClientInteractionAgent:
   ➔ First point of contact for the traveler (client). Captures incident details via multilingual chat or voice, identifies the client and policy using name, ID, or geolocation, and triggers the triage process with a unique case ID. Tech: NLP, Google Cloud TTS, contextual empathy prompts.

2. 🤖 TriageMedicalAssessmentAgent:
   ➔ Classifies the case based on urgency and symptoms. Evaluates symptom severity using clinical rule sets, determines care level (outpatient, ER, inpatient), and escalates life-threatening cases to the Repatriation Agent. Tech: Decision trees, symptom checkers, rule-based protocols.

3. 🤖 ProviderNetworkAgent:
   ➔ Finds suitable nearby medical facilities based on location, language, specialty, and availability. Sends appointment requests, confirms bookings, and logs provider responses with estimated wait times. Tech: Fuzzy location matching, mocked RAG for provider directories.

4. 🤖 MedicalDocumentationAgent:
   ➔ Collects, translat

# 🧠 SECTION 2: Prompt Templates with Persona and Format


In [4]:
# ----------------------------------------
# 🧠 SECTION 2: Prompt Templates with Persona and Format + Enhancements
# ----------------------------------------

from langchain_core.prompts import ChatPromptTemplate
from collections import OrderedDict
import json
import yaml

# Define enhanced prompts with SSML and persona
prompt_templates = OrderedDict()

prompt_templates['ClientInteractionAgent'] = ChatPromptTemplate.from_template(
    """<speak>
        <prosody rate="medium" pitch="+2st">
            You are a friendly and empathetic insurance assistant.
            • Greet the traveler warmly.
            • Collect the following information:
                - Symptoms
                - Current location (city and country)
                - Personal identifiers (name or ID)
                - Travel dates
            • Use NLP to infer urgency and classify the case:
                - outpatient
                - emergency
            • Generate a unique case ID and trigger triage.
            • Output Format: JSON with fields: case_id, name, symptoms, location, urgency, classification.
        </prosody>
    </speak>"""
)

prompt_templates['TriageMedicalAssessmentAgent'] = ChatPromptTemplate.from_template(
    """You are a precise and empathetic triage assistant.
    • Evaluate the symptoms and medical history using clinical rules.
    • Classify urgency: outpatient, ER, or inpatient.
    • Escalate directly to Repatriation Agent for life-threatening cases.
    • Ask if symptoms began before the trip.
    • Output Format: JSON with fields: urgency, recommended_care, pre_existing_flag, escalate_flag."""
)

prompt_templates['ProviderNetworkAgent'] = ChatPromptTemplate.from_template(
    """You are a provider network specialist helping travelers find care.
    • Find the best matching provider based on:
        - Location
        - Specialty
        - Language
        - Safety rating
    • Query using: `hospital_network_lookup(location)`.
    • If a facility is blacklisted, trigger escalation.
    • Output Format: JSON with fields: hospital_name, address, specialty, safety_rating, blacklist_flag, contact_info."""
)

prompt_templates['MedicalDocumentationAgent'] = ChatPromptTemplate.from_template(
    """You are responsible for collecting and processing medical documents.
    • Request discharge summary, invoice, diagnostics, and Fit-to-Fly certificate.
    • Translate documents if not in the client's preferred language.
    • Extract diagnosis and treatment data.
    • Output Format: JSON with fields: report_status, fit_to_fly, diagnosis_summary, compliance_notes."""
)

prompt_templates['PolicyValidationAgent'] = ChatPromptTemplate.from_template(
    """You are the policy validation expert.
    • Validate coverage for the case using `policy_checker_tool`.
    • Check:
        - Incident type (accident/illness)
        - Coverage limits
        - Exclusions
        - Travel date validity
        - Blacklisted providers
    • Output Format: JSON with fields: is_covered, exclusions, incident_type, validation_notes, blacklisted_provider."""
)

prompt_templates['RepatriationPlannerAgent'] = ChatPromptTemplate.from_template(
    """You plan medical repatriation for international travelers.
    • Choose optimal transport: air ambulance, stretcher, WCHC/WCHR/WCHS, escort (nurse/doctor), ground transport.
    • If escort or Level 3 country, notify ACC immediately.
    • Output Format: JSON with fields: transport_mode, escort_required, acc_notified, fit_to_fly_required, questionnaire_sent."""
)

prompt_templates['MedicalDecisionAgent'] = ChatPromptTemplate.from_template(
    """You are simulating the medical team's judgment.
    • Review diagnosis and treatment plan.
    • Approve, revise, or escalate based on clinical appropriateness.
    • Consult ACC for Level 3 care or high-risk profiles.
    • Output Format: JSON with fields: decision, notes, escalate_flag, approved_facility."""
)

prompt_templates['ComplianceConsentAgent'] = ChatPromptTemplate.from_template(
    """You ensure GDPR and HIPAA compliance during the case.
    • Confirm that the client has consented to:
        - Data sharing with hospitals and ACC
        - Repatriation arrangements
    • Generate encrypted approval log using `Fernet`.
    • Output Format: JSON with fields: consent_granted, timestamp, encrypted_log_key, jurisdiction."""
)

prompt_templates['CountryCareLevelAgent'] = ChatPromptTemplate.from_template(
    """You assign the care level of the client's current country.
    • Levels:
        - Level 1: High quality
        - Level 2: Moderate
        - Level 3: Low (trigger escalation if admitted)
    • Notify ACC and log to DCR tracker if Level 3 and admitted.
    • Output Format: JSON with fields: care_level, notify_paris_acc, dcr_logged, msc_contact_due."""
)

prompt_templates['OrchestratorAgent'] = ChatPromptTemplate.from_template(
    """You orchestrate the entire case workflow.
    • Route the case through the following agents in sequence:
        - ClientInteraction → Triage → Provider → Docs → Policy → Medical Decision → Repatriation → Consent
    • Monitor agent timing, failure states, and escalation points.
    • Log progress to the KPI dashboard and simulate human-AI comparison.
    • Output Format: JSON with fields: completed_steps, timing_stats, escalation_flags, ab_test_summary."""
)

# ------------------------------------------------
# 📤 EXPORTS: JSON and YAML for UI or config usage
# ------------------------------------------------

# Extract the actual prompt strings
json_data = {
    k: v.messages[0].prompt.template
    for k, v in prompt_templates.items()
}

# Export to JSON
with open("agent_prompts.json", "w") as json_file:
    json.dump(json_data, json_file, indent=2)

# Export to YAML
with open("agent_prompts.yaml", "w") as yaml_file:
    yaml.dump(json_data, yaml_file, sort_keys=False)

# ------------------------------------------------
# ⚙️ LangGraph-Compatible Prompt Wrappers
# ------------------------------------------------
def create_agent_node(agent_name):
    def agent_node(state):
        prompt = prompt_templates[agent_name].format_messages(**state)
        # You could also add TTS generation or API calls here
        return {"response": prompt}
    return agent_node

# Example usage:
client_interaction_node = create_agent_node("ClientInteractionAgent")
triage_node = create_agent_node("TriageMedicalAssessmentAgent")



# ----------------------------------------
# 🧠 SECTION 3: Phases 1–3 + Multi-Patient + Full Playback + PDF Export + OpenAI Key Fix
# ----------------------------------------

In [5]:
pip install gTTS


Note: you may need to restart the kernel to use updated packages.


In [6]:
import os

# Make sure the sounds/ directory exists
os.makedirs("sounds", exist_ok=True)


In [7]:
import os
import shutil

# Step 1: Create the target directory if it doesn't exist
os.makedirs("sounds", exist_ok=True)

# Step 2: Copy the ringtone from Kaggle input to working directory
source_path = "/kaggle/input/ringtone/phone-ringtone-telephone-324474.mp3"
target_path = "sounds/ringtone.mp3"

# Step 3: Copy the file
shutil.copy(source_path, target_path)

print(f"✅ Ringtone copied to: {target_path}")



✅ Ringtone copied to: sounds/ringtone.mp3


In [8]:
# ----------------------------------------
# 🧠 ENHANCED SECTION 3: Phases 1–3 + Multi-Patient + Full Playback + PDF Export + OpenAI Key Fix
# ----------------------------------------

from pathlib import Path
from pydub import AudioSegment
from google.cloud import texttospeech
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import gradio as gr
import os, random, json, datetime

# ----------------------------------------
# 🔐 Load OpenAI API Key from Kaggle Secrets
# ----------------------------------------

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("OPENAI_API_KEY")
except Exception as e:
    print("❌ Failed to load OPENAI_API_KEY:", e)

# ----------------------------------------
# 👥 Multi-Patient Support
# ----------------------------------------

def get_patient_by_name(name):
    patients = {
        "anne": {
            "name": "Anne",
            "location": "Nice, France",
            "symptoms": "severe leg pain after a fall",
            "urgency": "emergency"
        },
        "liam": {
            "name": "Liam",
            "location": "Da Nang, Vietnam",
            "symptoms": "high fever and dizziness",
            "urgency": "outpatient"
        },
        "priya": {
            "name": "Priya",
            "location": "Doha Airport, Qatar",
            "symptoms": "abdominal pain",
            "urgency": "emergency"
        }
    }
    return patients.get(name.lower())

# ----------------------------------------
# 🎭 PHASE 1: Emotion presets
# ----------------------------------------

agent_emotions = {
    "ClientAgent": "stress",
    "ClientAgent_2": "stress",
    "ClientAgent_3": "concerned",
    "ClientAgent_4": "curious",
    "ClientAgent_5": "in_pain",
    "ClientAgent_6": "grateful",
    "ClientAgent": "stress",
    "ClientInteractionAgent": "calm",
    "TriageMedicalAssessmentAgent": "urgent",
    "ProviderNetworkAgent": "neutral",
    "PolicyValidationAgent": "neutral",
    "MedicalDocumentationAgent": "calm",
    "RepatriationPlannerAgent": "calm",
    "MedicalDecisionAgent": "calm",
    "ComplianceConsentAgent": "neutral",
    "CountryCareLevelAgent": "neutral",
    "OrchestratorAgent": "calm"
}

audio_dir = Path("tts_audio"); audio_dir.mkdir(exist_ok=True)
log_file = Path("case_log.txt")
zip_output = Path("case_export.zip")

ambient_map = {
    "hospital": "ambient_hospital.mp3",
    "airport": "ambient_airport.mp3"
}

try:
    tts_client = texttospeech.TextToSpeechClient()
except Exception as e:
    print("⚠️ Google TTS setup failed:", e)
    tts_client = None

# ----------------------------------------
# 🔈 SSML-based TTS with 📞 Ringtone Support
# ----------------------------------------

from gtts import gTTS
from pydub import AudioSegment
import io

# Toggle between Google Cloud TTS and gTTS
use_google_tts = False

from gtts import gTTS
from pydub import AudioSegment
import io

# Optional ringtone path (5s slice)
ringtone_path = Path("sounds/ringtone.mp3")
ringtone = AudioSegment.from_file(ringtone_path)[:5000] if ringtone_path.exists() else AudioSegment.silent(duration=5000)

# Voice presets
client_voices = {
    "liam": "en-GB-Standard-A",   # ✅ Deep British male
    "anne": "en-GB-Wavenet-F",    # Female
    "priya": "en-GB-Wavenet-F"    # Female
}
agent_voice = "en-GB-Wavenet-D"  # Neutral/friendly female support voice

def synthesize_speech(text, agent, emotion="neutral", context="none"):
    pitch = "+2st" if emotion == "calm" else "-2st"
    rate = "slow" if emotion == "stress" else "medium"
    if emotion == "urgent":
        rate = "fast"
        pitch = "+0st"

    mp3_path = audio_dir / f"{agent}_{random.randint(1000, 9999)}.mp3"

    # 📞 Detect ringtone
    has_ringtone = "📞" in text
    clean_text = text.replace("📞", "").strip()

    # 👥 Determine speaker role
    is_client = agent.startswith("ClientAgent")

    # 👥 Assign voice based on speaker role and content
    if is_client:
        if "anne" in clean_text.lower():
            voice_name = "en-GB-Wavenet-F"
        elif "priya" in clean_text.lower():
            voice_name = "en-GB-Wavenet-F"
        else:
            voice_name = "en-GB-Wavenet-B"  # Male voice for Liam
    else:
        voice_name = "en-GB-Wavenet-F"  # Female voice for agents

    print(f"🎤 Using voice {voice_name} for agent '{agent}'")

    # Try to extract patient name from the sentence for dynamic mapping
    for name in client_voices.keys():
        if name.lower() in clean_text.lower():
            patient_name = name.lower()
            break

    # 🎤 Select voice
    # Set patient name safely (fallback to 'liam' if not found)
    patient_name = ""
    if "liam" in clean_text.lower():
        patient_name = "liam"
    elif "anne" in clean_text.lower():
        patient_name = "anne"
    elif "priya" in clean_text.lower():
        patient_name = "priya"
    else:
        patient_name = "liam"  # Default

    # Choose voice
    voice_name = client_voices.get(patient_name, "en-GB-Wavenet-B") if is_client else agent_voice

    print(f"🎙️ Speaker: {agent} → Voice: {voice_name}")

    try:
        if use_google_tts and tts_client:
            # Google Cloud TTS
            ssml = f"""
            <speak>
              <prosody rate="{rate}" pitch="{pitch}">
                {clean_text}
              </prosody>
            </speak>
            """
            input_text = texttospeech.SynthesisInput(ssml=ssml)
            voice = texttospeech.VoiceSelectionParams(language_code="en-GB", name=voice_name)
            audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

            response = tts_client.synthesize_speech(input=input_text, voice=voice, audio_config=audio_config)
            voice_audio = AudioSegment.from_file(io.BytesIO(response.audio_content), format="mp3")

        else:
            # gTTS fallback
            print(f"🗣️ Using gTTS fallback for {agent}")
            tts = gTTS(text=clean_text, lang="en", slow=False)
            temp_path = audio_dir / f"temp_{agent}.mp3"
            tts.save(temp_path)
            voice_audio = AudioSegment.from_file(temp_path)
            temp_path.unlink()

        # 🔔 Ringtone + pause
        pause = AudioSegment.silent(duration=700)
        final_audio = ringtone + pause + voice_audio if has_ringtone else voice_audio
        final_audio.export(mp3_path, format="mp3")
        return str(mp3_path)

    except Exception as e:
        print(f"❌ TTS failed for {agent}: {e}")
        return generate_placeholder_audio(agent, clean_text)


def generate_placeholder_audio(agent, text=""):
    has_ringtone = "📞" in text
    silent = AudioSegment.silent(duration=1000)
    ring = ringtone[:5000] if has_ringtone else AudioSegment.silent(duration=0)
    final_audio = ring + silent
    path = audio_dir / f"NO_AUDIO_{agent}.mp3"
    final_audio.export(path, format="mp3")
    return str(path)


# ----------------------------------------
# 🧠 PHASE 3: Mocked RAG Knowledge Bases
# ----------------------------------------

Path("rag_docs").mkdir(exist_ok=True)
Path("rag_docs/hospital_data.txt").write_text(
    "Hospital Pasteur is a Level 1 trauma center in Nice, France. It includes ICU facilities and is in-network."
)
Path("rag_docs/policy_terms.txt").write_text(
    "Standard policy covers outpatient and emergency treatment, includes repatriation with escort in emergencies."
)

def create_rag_chain(file):
    loader = TextLoader(file)
    docs = loader.load()
    chunks = CharacterTextSplitter(chunk_size=300, chunk_overlap=50).split_documents(docs)
    vector = FAISS.from_documents(chunks, OpenAIEmbeddings())
    return RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0), retriever=vector.as_retriever())

rag_hospital = create_rag_chain("rag_docs/hospital_data.txt")
rag_policy = create_rag_chain("rag_docs/policy_terms.txt")

# ----------------------------------------
# 🔗 PHASE 2: LangGraph Agent Nodes (with schema + debugging)
# ----------------------------------------

from typing import TypedDict, List
from langgraph.graph import StateGraph

# ✅ 1. Define your state schema
class AgentState(TypedDict):
    patient: dict
    script: dict
    log: List[str]
    audio: List[str]

# ✅ 2. Define each agent node function with debug
def agent_node(agent_name):
    def run(state: AgentState) -> AgentState:
        print(f"🚀 Executing {agent_name}...")  # Debug: agent being run

        emotion = agent_emotions.get(agent_name, "neutral")
        context = "hospital" if "Hospital" in agent_name else "airport" if "Repatriation" in agent_name else "none"
        msg = state["script"].get(agent_name, f"{agent_name} is processing...")

        if agent_name == "ProviderNetworkAgent":
            print("📡 RAG query: hospital")
            msg = rag_hospital.run("What care level does Hospital Pasteur provide?")
        elif agent_name == "PolicyValidationAgent":
            print("📡 RAG query: policy")
            msg = rag_policy.run("Is repatriation with escort covered?")

        state["log"].append(f"{agent_name}: {msg}")
        print(f"📝 Log entry added for {agent_name}")

        audio = synthesize_speech(msg, agent=agent_name, emotion=emotion, context=context)
        print(f"🔊 Audio synthesized for {agent_name}: {audio}")

        state["audio"].append(audio)
        return state
    return run

# ✅ 3. Build the workflow graph using the schema
def build_workflow():
    print("🛠️ Building LangGraph workflow...")

    graph = StateGraph(AgentState)

    # ➕ Add all nodes
    nodes = list(agent_emotions.keys())
    for node in nodes:
        print(f"➕ Adding node: {node}")
        graph.add_node(node, agent_node(node))

    # 🔗 Add edges (INSERT YOUR EDGE LOGIC HERE)
    graph.add_edge("ClientAgent", "ClientInteractionAgent")
    graph.add_edge("ClientInteractionAgent", "TriageMedicalAssessmentAgent")
    graph.add_edge("TriageMedicalAssessmentAgent", "ClientAgent_2")
    graph.add_edge("ClientAgent_2", "ProviderNetworkAgent")
    graph.add_edge("ProviderNetworkAgent", "ClientAgent_3")
    graph.add_edge("ClientAgent_3", "MedicalDocumentationAgent")
    graph.add_edge("MedicalDocumentationAgent", "ClientAgent_4")
    graph.add_edge("ClientAgent_4", "PolicyValidationAgent")
    graph.add_edge("PolicyValidationAgent", "MedicalDecisionAgent")
    graph.add_edge("MedicalDecisionAgent", "ClientAgent_5")
    graph.add_edge("ClientAgent_5", "RepatriationPlannerAgent")
    graph.add_edge("RepatriationPlannerAgent", "ComplianceConsentAgent")
    graph.add_edge("ComplianceConsentAgent", "ClientAgent_6")
    graph.add_edge("ClientAgent_6", "CountryCareLevelAgent")
    graph.add_edge("CountryCareLevelAgent", "OrchestratorAgent")

    # 🚀 Set entry/finish points
    graph.set_entry_point("ClientAgent")
    graph.set_finish_point("OrchestratorAgent")

    compiled_graph = graph.compile()
    print("✅ LangGraph compiled successfully!")
    return compiled_graph
    
# ----------------------------------------
# 🧩 Combine All Audio Clips
# ----------------------------------------

def concatenate_audio(audio_paths, output_path):
    combined = AudioSegment.empty()
    for path in audio_paths:
        combined += AudioSegment.from_file(path)
    combined.export(output_path, format="mp3")
    return output_path

# ----------------------------------------
# 📝 Generate PDF Conversation Log
# ----------------------------------------

def generate_pdf_from_log(log_lines, pdf_path):
    c = canvas.Canvas(str(pdf_path), pagesize=letter)
    width, height = letter
    c.setFont("Helvetica", 10)
    c.drawString(30, height - 40, f"Conversation Log - Generated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    y = height - 60
    for line in log_lines:
        if y < 40:
            c.showPage(); c.setFont("Helvetica", 10); y = height - 40
        c.drawString(30, y, line)
        y -= 14
    c.save()

# ----------------------------------------
# ▶️ Run Simulation for Any Patient
# ----------------------------------------

# 💬 Patient-specific scripts
patient_scripts = {
    "Liam": {
        "ClientAgent": "📞 Hello? I’m Liam, I fell while hiking in Da Nang and my leg really hurts. I can't walk.",
        "ClientInteractionAgent": "Hi Liam. Are you alone? Do you have insurance details and your location?",
        "TriageMedicalAssessmentAgent": "This is an emergency. Dispatching ambulance. Don’t move.",
        "ClientAgent_2": "📞 Ambulance took me to a clinic. I’m not sure it’s safe. Can you advise?",
        "ProviderNetworkAgent": "Please go to Hospital Pasteur. It’s in our network and has English-speaking doctors.",
        "ClientAgent_3": "📞 I’m here. Can you contact the doctor? I need advice.",
        "MedicalDocumentationAgent": "We’ll request your medical report and Fit-to-Fly if you’re discharged.",
        "ClientAgent_4": "📞 Can my partner travel with me back home?",
        "PolicyValidationAgent": "If they are listed, yes. Repatriation is covered for you.",
        "MedicalDecisionAgent": "The injury is stable. You’re getting proper care.",
        "ClientAgent_5": "📞 Discharged but still in pain. I need support to fly.",
        "RepatriationPlannerAgent": "We’ll arrange wheelchair assistance and extra seat for your leg.",
        "ComplianceConsentAgent": "Do you consent to share your medical data for travel planning?",
        "ClientAgent_6": "📞 Yes, I consent.",
        "CountryCareLevelAgent": "Vietnam is Level 2. No escalation needed.",
        "OrchestratorAgent": "Case closed for Liam. Logs and KPIs updated."
    },
    "Anne": {
        "ClientAgent": "📞 Hi, this is Anne. I slipped at my hotel in Rome. I think I fractured my arm.",
        "ClientInteractionAgent": "Hi Anne. I’m here to help. Can you describe your symptoms and location?",
        "TriageMedicalAssessmentAgent": "This may require an ER visit. Let’s send a doctor.",
        "ClientAgent_2": "📞 I'm at the clinic but unsure if it’s reliable.",
        "ProviderNetworkAgent": "Go to Policlinico Umberto I. It’s trusted and has English-speaking staff.",
        "ClientAgent_3": "📞 Doctor saw me. Can you request the documents?",
        "MedicalDocumentationAgent": "Getting discharge report and invoice. Requesting Fit-to-Fly if needed.",
        "ClientAgent_4": "📞 What’s covered under my policy?",
        "PolicyValidationAgent": "Treatment is covered. Repatriation too if you’re unable to travel alone.",
        "MedicalDecisionAgent": "Fracture confirmed. Non-surgical. Safe for return with escort.",
        "ClientAgent_5": "📞 I’m in a sling. It’s hard to carry luggage.",
        "RepatriationPlannerAgent": "We’ll arrange a nurse escort and assistance throughout the journey.",
        "ComplianceConsentAgent": "Do you consent to share reports with airline and our team?",
        "ClientAgent_6": "📞 Yes, I consent.",
        "CountryCareLevelAgent": "Italy is Level 1. Standard follow-up applies.",
        "OrchestratorAgent": "Case closed for Anne. Everything logged."
    },
    "Priya": {
        "ClientAgent": "📞 Hello, I’m Priya. I’ve had food poisoning in Bangkok and feel very weak.",
        "ClientInteractionAgent": "Hi Priya. I’m sorry to hear that. Let’s get your location and insurance ID.",
        "TriageMedicalAssessmentAgent": "This might be outpatient. We’ll send a doctor to your hotel.",
        "ClientAgent_2": "📞 The doctor came but now I’m worse.",
        "ProviderNetworkAgent": "Switch to Bumrungrad Hospital – top-rated with translators on staff.",
        "ClientAgent_3": "📞 I’m at the ER now. What’s next?",
        "MedicalDocumentationAgent": "We’re retrieving your reports and confirming Fit-to-Fly readiness.",
        "ClientAgent_4": "📞 I’m flying soon. Will this affect my coverage?",
        "PolicyValidationAgent": "Yes, but outpatient care is covered. Flight may need rebooking.",
        "MedicalDecisionAgent": "Symptoms under control. OK to fly with precautions.",
        "ClientAgent_5": "📞 Still feeling dizzy.",
        "RepatriationPlannerAgent": "We’ll book a business class seat and ground escort to the airport.",
        "ComplianceConsentAgent": "Do we have your consent to proceed?",
        "ClientAgent_6": "📞 Yes, go ahead.",
        "CountryCareLevelAgent": "Thailand is Level 2. Monitoring continues.",
        "OrchestratorAgent": "Priya’s case wrapped up. Logs completed."
    }
}


def run_simulation_ui(patient_name):
    patient = get_patient_by_name(patient_name)
    if not patient:
        return "❌ Patient not found.", None, None

    # 🧠 Select patient-specific script
    script = patient_scripts.get(patient_name.lower().capitalize())
    if not script:
        return "❌ No conversation script found for this patient.", None, None

    # 🧹 Cleanup previous logs/audio
    if log_file.exists():
        log_file.unlink()
    for f in audio_dir.glob("*.mp3"):
        f.unlink()

    # 🚀 Run workflow
    graph = build_workflow()
    state = graph.invoke({
        "patient": patient,
        "script": script,
        "log": [],
        "audio": []
    })

    # 🎧 Output paths
    full_audio_path = audio_dir / f"{patient_name}_full_convo.mp3"
    pdf_path = audio_dir / f"{patient_name}_conversation.pdf"

    concatenate_audio(state["audio"], full_audio_path)
    generate_pdf_from_log(state["log"], pdf_path)

    # 📦 Export ZIP
    with zip_output.open("wb") as f:
        from zipfile import ZipFile
        with ZipFile(f, "w") as zipf:
            for a in state["audio"]:
                zipf.write(a, arcname=os.path.basename(a))
            with open(log_file, "w") as lf:
                lf.write("\n".join(state["log"]))
            zipf.write(log_file, arcname=log_file.name)
            zipf.write(pdf_path, arcname=pdf_path.name)
            zipf.write(full_audio_path, arcname=full_audio_path.name)

    return "\n".join(state["log"]), str(zip_output), str(full_audio_path)

# ----------------------------------------
# 🎛️ Gradio UI with Full Playback + PDF Export
# ----------------------------------------

def launch_ui():
    gr.Interface(
        fn=run_simulation_ui,
        inputs=gr.Dropdown(choices=["Anne", "Liam", "Priya"], label="Select Patient"),
        outputs=[
            gr.Textbox(label="Conversation Log"),
            gr.File(label="Download ZIP (Logs + Audio + PDF)"),
            gr.Audio(label="Listen to Full Conversation", type="filepath", show_download_button=True)
        ],
        title="🧠 Global MedAssist – Full Playback + PDF",
        description="Multi-agent simulation with SSML tone, LangGraph, RAG, and PDF export"
    ).launch(share=True)

# 🔥 Launch it
launch_ui()


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://bff754bb1cd92b7d64.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## SECTION 4: Unit Test & Debugging Code:

In [9]:
import unittest
import os
import json
from pathlib import Path
from datetime import datetime
from geopy.distance import distance

# Define the functions being tested

def generate_case_number():
    return f"GB1-{random.randint(100, 999)}-{random.randint(100, 999)}"

def save_case(case_data):
    case_number = case_data["case_number"]
    with open(f"case_files/{case_number}.json", "w") as f:
        json.dump(case_data, f, indent=4)

def extract_coordinates_from_text(text):
    coords = re.findall(r"(-?\d+\.\d+),\s*(-?\d+\.\d+)", text)
    if coords:
        return float(coords[0][0]), float(coords[0][1])
    return None

def find_closest_hospital(user_coords):
    min_dist = float("inf")
    closest = None
    for name, coords in hospital_locations.items():
        dist_km = distance(user_coords, coords).km
        if dist_km < min_dist:
            min_dist = dist_km
            closest = (name, coords, dist_km)
    return closest

# Test cases

class TestMedicalAssistantFunctions(unittest.TestCase):

    def test_generate_case_number(self):
        # Test case number generation
        case_number = generate_case_number()
        self.assertTrue(case_number.startswith("GB1-"))
        self.assertEqual(len(case_number.split('-')[1]), 3)  # Checks the length of the first random part
        self.assertEqual(len(case_number.split('-')[2]), 3)  # Checks the length of the second random part

    def test_save_case(self):
        # Test saving case data to file
        case_data = {
            "case_number": "GB1-123-456",
            "full_name": "John Doe",
            "home_address": "123 Main St",
            "outbound_flight": "2025-04-01",
            "return_flight": "2025-04-10",
            "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "chat_history": []
        }
        save_case(case_data)  # Saving case
        self.assertTrue(Path(f"case_files/{case_data['case_number']}.json").exists())  # Check if file is saved

    def test_extract_coordinates_from_text(self):
        # Test extracting coordinates from a string
        text = "The hospital is located at 13.7489, 100.5562."
        coords = extract_coordinates_from_text(text)
        self.assertEqual(coords, (13.7489, 100.5562))  # Test if the extracted coordinates match

    def test_find_closest_hospital(self):
        # Test finding the closest hospital
        coords = (13.7489, 100.5562)
        closest_hospital = find_closest_hospital(coords)
        self.assertTrue(closest_hospital[2] < 100)  # Check if the distance is below a certain threshold

# Run tests
def run_tests():
    # Initialize test suite
    suite = unittest.TestLoader().loadTestsFromTestCase(TestMedicalAssistantFunctions)
    unittest.TextTestRunner().run(suite)

run_tests()




EE.E
ERROR: test_extract_coordinates_from_text (__main__.TestMedicalAssistantFunctions.test_extract_coordinates_from_text)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_31/202697182.py", line 62, in test_extract_coordinates_from_text
    coords = extract_coordinates_from_text(text)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_31/202697182.py", line 19, in extract_coordinates_from_text
    coords = re.findall(r"(-?\d+\.\d+),\s*(-?\d+\.\d+)", text)
             ^^
NameError: name 're' is not defined

ERROR: test_find_closest_hospital (__main__.TestMedicalAssistantFunctions.test_find_closest_hospital)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_31/202697182.py", line 68, in test_find_closest_hospital
    closest_hospital = find_closest_hospital(coords)
                       ^^^^^^^^^^^^^^^^^^^^^^

# SECTION 5: Travel Assistance Chat bot

In [10]:
!pip install folium --quiet
!pip install transformers torchvision torch --quiet
!pip install python-docx fpdf --quiet
!pip install gradio --quiet
!pip install fitz --quiet
!pip install tools --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have 

# SECTION 5.1. Generating Medical Reports

In [11]:
from docx import Document
from fpdf import FPDF
import os
import json

# Directory to store fake reports on Kaggle
output_dir = "/kaggle/working/medical_reports"
os.makedirs(output_dir, exist_ok=True)

# Detailed fake medical reports
detailed_medical_reports = {
    "Liam": {
        "patient_name": "Liam Thompson",
        "dob": "1990-07-15",
        "admission_date": "2025-04-13",
        "discharge_date": "2025-04-17",
        "incident_description": "Patient slipped while hiking in Da Nang, landed on his right leg with a twisting force.",
        "vitals_on_admission": {
            "BP": "135/85 mmHg",
            "HR": "88 bpm",
            "Temp": "37.2°C",
            "RR": "18 breaths/min",
            "SpO2": "98% on room air"
        },
        "diagnosis": "Closed displaced comminuted fracture of the mid-shaft of the right tibia.",
        "tests": {
            "X-ray": "Fracture confirmed with slight displacement, no fibular involvement.",
            "CBC": "WBC: 8.2 x10^9/L, Hb: 14.5 g/dL, Platelets: 210 x10^9/L",
            "CRP": "Normal"
        },
        "treatment": "Leg immobilized with fiberglass cast under sedation. Analgesics administered: IV Paracetamol and Morphine. Antibiotic prophylaxis given. Scheduled physiotherapy initiated.",
        "medications_received": [
            "IV Paracetamol 1g q8h",
            "IV Morphine 2mg PRN",
            "IV Ceftriaxone 1g once (prophylaxis)"
        ],
        "medical_evolution": "Stable vitals throughout admission. Pain managed effectively. Ambulation with walker initiated on Day 3.",
        "surgical_procedures": "None performed. Orthopedic review confirmed non-surgical management appropriate.",
        "discharge_medications": [
            "Paracetamol 500mg PO q6h PRN",
            "Ibuprofen 400mg PO TID"
        ],
        "equipment_on_discharge": "Full-length leg cast with instructions for non-weight bearing.",
        "fit_to_fly": "Yes, with wheelchair assistance and extra legroom.",
        "prognosis": "Favorable recovery expected within 6–8 weeks.",
        "recommendation": "Repatriation with commercial flight, nurse escort not necessary.",
        "discharge_plan": "Follow-up in orthopedic clinic in home country in 7 days. Continue analgesics and physiotherapy exercises."
    },
    "Anne": {
        "patient_name": "Anne Dupont",
        "dob": "1987-11-23",
        "admission_date": "2025-04-10",
        "discharge_date": "2025-04-12",
        "incident_description": "Slipped on wet bathroom floor in hotel, landed on outstretched left hand.",
        "vitals_on_admission": {
            "BP": "120/80 mmHg",
            "HR": "75 bpm",
            "Temp": "36.8°C",
            "RR": "16 breaths/min",
            "SpO2": "99%"
        },
        "diagnosis": "Non-displaced hairline fracture of the distal left radius.",
        "tests": {
            "X-ray": "Confirmed distal radial fracture with no displacement or angulation.",
            "CBC": "Normal",
            "Electrolytes": "Normal"
        },
        "treatment": "Arm placed in a padded sling. No reduction required. Pain managed conservatively.",
        "medications_received": [
            "Oral Ibuprofen 400mg TID",
            "Oral Paracetamol 500mg PRN"
        ],
        "medical_evolution": "No swelling progression. Pain reduced after 48h. No complications.",
        "surgical_procedures": "Not indicated.",
        "discharge_medications": [
            "Paracetamol 500mg PO q6h PRN"
        ],
        "equipment_on_discharge": "Sling to immobilize left arm.",
        "fit_to_fly": "Yes, sling use and baggage assistance required.",
        "prognosis": "Expected full recovery in 4–5 weeks with outpatient follow-up.",
        "recommendation": "Repatriation with nurse escort to assist with mobility and baggage.",
        "discharge_plan": "Orthopedic follow-up in 10 days. Avoid weight bearing with left hand."
    },
    "Priya": {
        "patient_name": "Priya Mehta",
        "dob": "1995-02-02",
        "admission_date": "2025-04-09",
        "discharge_date": "2025-04-11",
        "incident_description": "Consumed seafood at local night market in Bangkok, followed by vomiting and diarrhea.",
        "vitals_on_admission": {
            "BP": "100/65 mmHg",
            "HR": "105 bpm",
            "Temp": "38.1°C",
            "RR": "20 breaths/min",
            "SpO2": "98%"
        },
        "diagnosis": "Acute gastroenteritis with moderate dehydration.",
        "tests": {
            "Stool culture": "Pending",
            "CBC": "WBC: 12.3 x10^9/L, Hb: 13.0 g/dL",
            "Electrolytes": "Na: 130 mmol/L, K: 3.2 mmol/L"
        },
        "treatment": "Admitted for IV fluid replacement. Antiemetics and broad-spectrum antibiotics administered.",
        "medications_received": [
            "IV Ringer’s Lactate",
            "Ondansetron 4mg IV",
            "Oral Ciprofloxacin 500mg BID x3 days"
        ],
        "medical_evolution": "Improved hydration, no further vomiting after Day 1. Oral intake resumed.",
        "surgical_procedures": "None.",
        "discharge_medications": [
            "Oral Rehydration Salts",
            "Ciprofloxacin 500mg BID (complete 3-day course)"
        ],
        "equipment_on_discharge": "None required.",
        "fit_to_fly": "Yes, after 48-hour monitoring and electrolyte correction.",
        "prognosis": "Full recovery expected in 2–3 days.",
        "recommendation": "Repatriation by commercial flight, no escort required.",
        "discharge_plan": "Continue oral hydration and antibiotics. Follow-up only if symptoms return."
    }
}

# Save JSON for app use
json_path = os.path.join(output_dir, "summary_reports.json")
with open(json_path, "w") as f:
    json.dump(detailed_medical_reports, f, indent=4)

# Sanitizer for special characters
def sanitize_text(text):
    if isinstance(text, str):
        return (
            text.replace("–", "-")
                .replace("—", "-")
                .replace("’", "'")
                .replace("‘", "'")
                .replace("“", '"')
                .replace("”", '"')
                .encode("latin-1", "ignore").decode("latin-1")
        )
    return str(text)

# Generate both PDF and DOCX
def generate_reports():
    paths = []
    for person, data in detailed_medical_reports.items():
        # PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        for key, value in data.items():
            if isinstance(value, dict):
                pdf.cell(200, 10, txt=sanitize_text(f"{key.replace('_', ' ').title()}: "), ln=True)
                for sub_key, sub_val in value.items():
                    pdf.cell(200, 10, txt=sanitize_text(f"  {sub_key}: {sub_val}"), ln=True)
            elif isinstance(value, list):
                pdf.cell(200, 10, txt=sanitize_text(f"{key.replace('_', ' ').title()}: "), ln=True)
                for item in value:
                    pdf.cell(200, 10, txt=sanitize_text(f"  - {item}"), ln=True)
            else:
                pdf.cell(200, 10, txt=sanitize_text(f"{key.replace('_', ' ').title()}: {value}"), ln=True)
        pdf_path = os.path.join(output_dir, f"{person}_report.pdf")
        pdf.output(pdf_path)
        paths.append(pdf_path)

        # DOCX
        doc = Document()
        doc.add_heading(f"Medical Report - {person}", 0)
        for key, value in data.items():
            if isinstance(value, dict):
                doc.add_heading(key.replace('_', ' ').title(), level=2)
                for sub_key, sub_val in value.items():
                    doc.add_paragraph(f"{sub_key}: {sub_val}")
            elif isinstance(value, list):
                doc.add_heading(key.replace('_', ' ').title(), level=2)
                for item in value:
                    doc.add_paragraph(f"- {item}")
            else:
                doc.add_paragraph(f"{key.replace('_', ' ').title()}: {value}")
        docx_path = os.path.join(output_dir, f"{person}_report.docx")
        doc.save(docx_path)
        paths.append(docx_path)

    return paths

# Run report generation
report_paths = generate_reports()
report_paths



['/kaggle/working/medical_reports/Liam_report.pdf',
 '/kaggle/working/medical_reports/Liam_report.docx',
 '/kaggle/working/medical_reports/Anne_report.pdf',
 '/kaggle/working/medical_reports/Anne_report.docx',
 '/kaggle/working/medical_reports/Priya_report.pdf',
 '/kaggle/working/medical_reports/Priya_report.docx']

# SECTION 5.2. Refactoring Section: Synchronizing Agents with the LLM

This section introduces the necessary changes to synchronize all agents with the LLM (OpenAI GPT-4) for medical case management. It includes a utility function for querying the LLM, modified agent functions to communicate with the LLM, and orchestration logic to handle data flow between agents.

The query_llm function centralizes LLM communication, making it easier for agents to query the LLM without repeating code.

Each agent now passes its data to the LLM to receive contextually relevant responses.

The OrchestratorAgent ensures that the workflow between agents is synchronized and maintains a unified chat history across the entire case management process.

This refactoring ensures that all agents interact with the LLM during their tasks. Each agent queries the LLM with relevant data, which ensures synchronized decision-making, empathetic responses, and a streamlined case management process.

In [20]:
# Utility function to handle communication with the LLM
def query_llm(agent_name, input_data, chat_history=[]):
    """
    Sends a query to the LLM (OpenAI GPT-4) and returns the response.

    Args:
    - agent_name (str): The name of the agent requesting the information.
    - input_data (str): The data or query to send to the LLM.
    - chat_history (list): The conversation history to maintain context.

    Returns:
    - str: The LLM-generated response.
    """
    # Prepares the system message (guiding the agent's behavior)
    system_prompt = {
        "role": "system",
        "content": f"You are the {agent_name}, an AI agent in a travel health insurance system. Provide thoughtful, empathetic, and accurate responses."
    }

    # Structure the LLM prompt based on the agent's task
    user_prompt = {
        "role": "user",
        "content": input_data
    }

    # Complete messages list
    messages = [system_prompt] + chat_history + [user_prompt]

    # Query the OpenAI LLM
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages,
        max_tokens=700
    )

    # Extract and return the LLM's response
    return response.choices[0].message.content


# Modify the agents to communicate with LLM

# Client Interaction Agent
def ClientInteractionAgent(client_data, chat_history=[]):
    input_data = f"Client details: Name: {client_data['full_name']}, Policy ID: {client_data['policy_id']}, Incident: {client_data['incident_details']}. What should be the next step?"
    response = query_llm("ClientInteractionAgent", input_data, chat_history)
    return response


# Triage Medical Assessment Agent
def TriageMedicalAssessmentAgent(symptoms, chat_history=[]):
    input_data = f"Symptoms reported: {symptoms}. Based on these, classify the case's urgency (outpatient, ER, inpatient)."
    response = query_llm("TriageMedicalAssessmentAgent", input_data, chat_history)
    return response


# Provider Network Agent
def ProviderNetworkAgent(location, medical_needs, chat_history=[]):
    input_data = f"Find nearby medical providers for location: {location} and needs: {medical_needs}. Provide a list of suitable hospitals."
    response = query_llm("ProviderNetworkAgent", input_data, chat_history)
    return response


# Medical Documentation Agent
def MedicalDocumentationAgent(document_data, chat_history=[]):
    input_data = f"Process the medical document: {document_data}. Summarize the key details for policy validation."
    response = query_llm("MedicalDocumentationAgent", input_data, chat_history)
    return response


# Policy Validation Agent
def PolicyValidationAgent(policy_data, treatment_details, chat_history=[]):
    input_data = f"Validate the policy coverage for the treatment: {treatment_details}. Check for exclusions or co-pays in policy: {policy_data}."
    response = query_llm("PolicyValidationAgent", input_data, chat_history)
    return response


# Medical Decision Agent
def MedicalDecisionAgent(medical_data, chat_history=[]):
    input_data = f"Analyze the medical case: {medical_data}. Provide a decision on the recommended treatment options."
    response = query_llm("MedicalDecisionAgent", input_data, chat_history)
    return response


# Compliance Consent Agent
def ComplianceConsentAgent(consent_data, chat_history=[]):
    input_data = f"Ensure that the client has accepted the data-sharing terms. Validate their consent for medical treatment in compliance with legal regulations."
    response = query_llm("ComplianceConsentAgent", input_data, chat_history)
    return response


# Country Care Level Agent
def CountryCareLevelAgent(country, chat_history=[]):
    input_data = f"Determine the risk level of the country: {country}. Trigger escalation if the client is in a Level 3 country."
    response = query_llm("CountryCareLevelAgent", input_data, chat_history)
    return response


# Repatriation Planner Agent
def RepatriationPlannerAgent(transport_needs, chat_history=[]):
    input_data = f"Plan the repatriation for the client. Assess the feasibility of commercial flights, air ambulances, or ground transport."
    response = query_llm("RepatriationPlannerAgent", input_data, chat_history)
    return response


# Orchestrator Agent to manage data flow between agents
def OrchestratorAgent(case_data, chat_history=[]):
    """
    Manages the flow of case data between the agents and keeps the conversation history intact.
    """
    for agent_name, agent_data in case_data.items():
        # Call the corresponding agent with the relevant data
        response = globals()[f"{agent_name}"](agent_data, chat_history)
        
        # Add the response to chat history for context
        chat_history.append({"role": "assistant", "content": response})
    
    # Return the final chat history for the entire process
    return chat_history


# SECTION 5.3. Imports and Global Setup for Medical Case Management

In [12]:
pip install PyMuPDF --quite



Usage:   
  /usr/bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /usr/bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /usr/bin/python3 -m pip install [options] [-e] <local project path> ...
  /usr/bin/python3 -m pip install [options] <archive url/path> ...

no such option: --quite
Note: you may need to restart the kernel to use updated packages.


In [18]:
import gradio as gr
from openai import OpenAI
from PIL import Image
import base64
import os
import folium
import re
import tempfile
from geopy.distance import distance
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import fitz  # PyMuPDF for reading PDFs
from docx import Document  # For Word documents
import json
from datetime import datetime
import random


client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Load Hugging Face model for image captioning
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# 🌍 Country Levels
level_1 = ["United States", "Canada", "United Kingdom", "Germany", "France", "Japan", "Australia", "New Zealand", "Sweden", "Norway", "Netherlands", "Switzerland", "Italy", "Spain", "South Korea", "Singapore"]
level_2 = ["Mexico", "Brazil", "Thailand", "Vietnam", "Turkey", "Malaysia", "Costa Rica", "Serbia", "India", "Philippines", "China", "Chile", "South Africa", "Indonesia", "Egypt", "UAE"]
level_3 = ["Nepal", "Kenya", "Nigeria", "Pakistan", "Bangladesh", "Ethiopia", "Uganda", "Myanmar", "Cameroon", "Burkina Faso", "Zimbabwe", "DR Congo", "Sudan", "Ghana", "Bolivia"]

# 🏥 Trusted Hospitals with coordinates
hospital_locations = {
    # Level 2 countries
    "Bumrungrad International Hospital": (13.7489, 100.5562),
    "Samitivej Hospital": (13.7300, 100.5684),
    "Hospital Pasteur": (16.0471, 108.2062),
    "Franco-Vietnamese Hospital": (10.7380, 106.7048),
    "Apollo Hospital": (12.9438, 77.5858),
    "Fortis Hospital": (28.4595, 77.0266),
    "Albert Einstein Hospital": (-23.6090, -46.6946),
    "Sírio-Libanês Hospital": (-23.5560, -46.6537),
    "Ángeles Hospital": (19.4326, -99.1332),
    "San Javier Hospital": (20.6736, -103.3442),
    "As-Salam International Hospital": (30.0444, 31.2357),
    "Cleopatra Hospital": (30.0571, 31.3199),
    "Siloam Hospitals": (-6.2088, 106.8456),
    "RSUP Dr. Sardjito": (-7.7684, 110.3786),
    "Aga Khan University Hospital": (-1.2921, 36.8219),
    "Nairobi Hospital": (-1.3000, 36.8000),
    "Lagoon Hospital": (6.5244, 3.3792),
    "Reddington Hospital": (6.4396, 3.4216),
    "Cleveland Clinic Abu Dhabi": (24.4539, 54.3773),
    "Mediclinic City Hospital": (25.2285, 55.3273),
    "Hanoi French Hospital": (21.003636, 105.840011),

    # Level 3 countries
    "Tribhuvan University Teaching Hospital": (27.7172, 85.3240),
    "Norvic International Hospital": (27.7060, 85.3171),
    "Mulago Hospital": (0.3365, 32.5825),
    "International Hospital Kampala": (0.3031, 32.5950),
    "Parirenyatwa General Hospital": (-17.8292, 31.0522),
    "Harare Central Hospital": (-17.8290, 31.0530),
    "Black Lion Hospital": (9.0326, 38.7468),
    "St. Paul's Hospital Millennium Medical College": (9.0176, 38.7498),
    "Korle Bu Teaching Hospital": (5.5400, -0.2237),
    "Nyaho Medical Centre": (5.6064, -0.1705),
    "BIRDEM General Hospital": (23.7380, 90.3948),
    "Square Hospital": (23.7520, 90.3776),
    "National Hospital Abuja": (9.0539, 7.4919),
    "University College Hospital Ibadan": (7.3878, 3.8966),
    "Indus Hospital Karachi": (24.8615, 67.0099),
    "Shifa International Hospital": (33.6938, 73.0652),
    "Yangon General Hospital": (16.7796, 96.1583),
    "Pun Hlaing Hospital": (16.8213, 96.1011),
    "Yaoundé Central Hospital": (3.8480, 11.5021),
    "Laquintinie Hospital": (4.0483, 9.7043),
    "CHU-YO (Ouagadougou)": (12.3615, -1.5339),
    "Polyclinique Notre Dame de la Paix": (12.3751, -1.5123),
    "General Hospital of Kinshasa": (-4.3276, 15.3136),
    "Ngaliema Clinic": (-4.3270, 15.3060),
    "Sudan Federal Hospital": (15.5007, 32.5599),
    "Al-Shaab Teaching Hospital": (15.5895, 32.5519),
    "Clinica Los Olivos": (-17.3926, -66.1605),
    "Hospital Univalle": (-17.3784, -66.1589)
}

# ... [no changes to get_country_level, extract_coordinates_from_text, find_closest_hospital, generate_map] ...
case_directory = "/kaggle/working/case_files"
os.makedirs(case_directory, exist_ok=True)

def generate_case_number():
    return f"GB1-{random.randint(100,999)}-{random.randint(100,999)}"

def initialize_case(full_name, home_address, outbound, inbound):
    case_number = generate_case_number()
    case = {
        "case_number": case_number,
        "full_name": full_name,
        "home_address": home_address,
        "outbound_flight": outbound,
        "return_flight": inbound,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "chat_history": []
    }
    with open(f"{case_directory}/{case_number}.json", "w") as f:
        json.dump(case, f, indent=4)
    return case

def load_case(case_number):
    path = f"{case_directory}/{case_number}.json"
    if os.path.exists(path):
        with open(path, "r") as f:
            return json.load(f)
    return None

def update_case(case_data):
    path = f"{case_directory}/{case_data['case_number']}.json"
    with open(path, "w") as f:
        json.dump(case_data, f, indent=4)

def get_country_level(country):
    if country in level_1:
        return "Level 1"
    elif country in level_2:
        return "Level 2"
    elif country in level_3:
        return "Level 3"
    return "Unknown"

def extract_coordinates_from_text(text):
    coords = re.findall(r"(-?\d+\.\d+),\s*(-?\d+\.\d+)", text)
    if coords:
        return float(coords[0][0]), float(coords[0][1])
    return None

def find_closest_hospital(user_coords):
    min_dist = float("inf")
    closest = None
    for name, coords in hospital_locations.items():
        dist_km = distance(user_coords, coords).km
        if dist_km < min_dist:
            min_dist = dist_km
            closest = (name, coords, dist_km)
    return closest

def generate_map(user_text):
    coords = extract_coordinates_from_text(user_text)
    if not coords:
        return "<p>No coordinates detected in message.</p>"
    closest = find_closest_hospital(coords)
    if not closest:
        return "<p>No hospital found nearby.</p>"
    hname, hcoords, dist = closest
    fmap = folium.Map(location=coords, zoom_start=10)
    folium.Marker(location=coords, popup="Client Location", icon=folium.Icon(color="blue")).add_to(fmap)
    folium.Marker(location=hcoords, popup=f"{hname} ({dist:.1f} km)", icon=folium.Icon(color="green")).add_to(fmap)
    tmp_file = tempfile.NamedTemporaryFile(suffix=".html", delete=False)
    fmap.save(tmp_file.name)
    with open(tmp_file.name, "r") as f:
        return f.read()

def generate_image_description(image_path):
    raw_image = Image.open(image_path).convert('RGB')
    inputs = processor(raw_image, return_tensors="pt")
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

def medical_chat(user_input, image=None, chat_history=[]):
    system_prompt = {
        "role": "system",
        "content": (
            "You are an experienced agent working in the Operations or Medical Team "
            "of a travel health insurance company. Respond empathetically and professionally. "
            "Above there is a JSON mapping of *all* our trusted hospitals worldwide, with their coordinates:"
            "Assess whether the hospital mentioned is in a trusted network and recommend next steps accordingly. "
            "Use the list of known countries and hospitals to guide your response. If unclear, ask questions to clarify."
        )
    }

    lower_input = user_input.lower()
    country_found = next((c for c in level_1 + level_2 + level_3 if c.lower() in lower_input), None)
    hospital_found = next((h for h in hospital_locations if h.lower() in lower_input), None)

    guidance = ""
    coords = extract_coordinates_from_text(user_input)
    if coords:
        closest = find_closest_hospital(coords)
        if closest:
            hname, hcoords, dist_km = closest
            dist_mi = dist_km * 0.621371
            transport = "an ambulance" if "severe" in lower_input or "bleeding" in lower_input or "can’t walk" in lower_input else "a taxi"
            maps_link = f"https://www.google.com/maps/dir/{coords[0]},{coords[1]}/{hcoords[0]},{hcoords[1]}"
            guidance = (
                f"🚑 Given your injury, it's crucial to seek care quickly. I recommend **{hname}**, "
                f"which is approximately **{dist_mi:.1f} miles** from your current location.\n\n"
                f"Please arrange for **{transport}** to take you there.\n\n"
                f"📍 [Click here for directions on Google Maps]({maps_link})\n\n"
                "Once you arrive, please confirm admission so we can begin coordinating follow-up care or repatriation if necessary."
            )
            chat_history.append({"role": "assistant", "content": guidance})

    elif country_found:
        level = get_country_level(country_found)
        if level == "Level 1":
            guidance = f"🟢 {country_found} is a Level 1 country. All hospitals are considered reliable."
        elif hospital_found:
            guidance = f"🟢 {hospital_found} in {country_found} is a trusted facility in our network. Care should be appropriate."
        else:
            guidance = (
                f"⚠️ {country_found} is a {level} country. If the hospital is not in our trusted network, "
                "we recommend moving the patient to a reliable facility or considering evacuation."
            )
        chat_history.append({"role": "assistant", "content": guidance})

    if image:
        image_caption = generate_image_description(image)
        chat_history.append({"role": "assistant", "content": f"🖼️ Injury analysis: {image_caption}"})
        if "deep" in image_caption or "open wound" in image_caption or "fracture" in image_caption:
            chat_history.append({"role": "assistant", "content": "⚠️ This injury appears serious. Immediate evaluation is required."})

    prompt_text = f"{user_input}\nImage description: {image_caption}" if image else user_input
    messages = [system_prompt] + chat_history + [{"role": "user", "content": prompt_text}]

    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            max_tokens=700
        )
        reply = response.choices[0].message.content
    except Exception as e:
        print(f"Error in medical chat: {e}")
        return "Error processing the request.", chat_history

    chat_history.append({"role": "user", "content": user_input})
    chat_history.append({"role": "assistant", "content": reply})
    return reply, chat_history

# --------------------- Onboarding + Case Rules --------------------------
def respond(message, image, chat_history, file, name, address, outbound, inbound, onboarded):
    if not onboarded:
        if not (name and address and outbound and inbound):
            return "❗Please complete onboarding.", chat_history, chat_history, "", onboarded, \
                   gr.update(visible=True), gr.update(visible=True), \
                   gr.update(visible=True), gr.update(visible=True), \
                   gr.update(visible=False)

        case = initialize_case(name, address, outbound, inbound)
        intro = f"✅ Welcome {name}. Your case number is {case['case_number']}. Please use this in future communication."
        chat_history.append({"role": "assistant", "content": intro})
        save_case(case)
        return "", chat_history, chat_history, "", True, \
               gr.update(visible=False), gr.update(visible=False), \
               gr.update(visible=False), gr.update(visible=False), \
               gr.update(visible=True, value=case['case_number'])

    file_text = ""
    if file:
        file_path = file.name
        file_text = extract_text_from_file(file_path)
        if file_text == "Unsupported file format.":
            chat_history.append({"role": "assistant", "content": "⚠️ I couldn't read the contents of the uploaded file. Please check the file format."})
        else:
            chat_history.append({"role": "user", "content": f"📄 Uploaded Medical Report:\n{file_text}"})

    reply, updated_history = medical_chat(message + "\n" + file_text, image, chat_history)
    map_html = generate_map(message)

    return "", updated_history, updated_history, map_html, onboarded, \
           gr.update(), gr.update(), gr.update(), gr.update(), gr.update()


# 🧠 Gradio UI
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="🖾 AI Health Assistant", type="messages")
    state = gr.State([])
    onboarded_state = gr.State(False)

    full_name = gr.Textbox(label="🧑 Full Name", visible=True)
    home_address = gr.Textbox(label="🏠 Home Address", visible=True)
    outbound_flight = gr.Textbox(label="📅 Outbound Flight Date (YYYY-MM-DD)", visible=True)
    return_flight = gr.Textbox(label="📅 Return Flight Date (YYYY-MM-DD)", visible=True)
    case_id_display = gr.Textbox(label="📁 Case Number", interactive=False, visible=False)

    with gr.Row():
        txt = gr.Textbox(label="💬 Your Message", placeholder="Describe your injury or ask a question...")
        img = gr.Image(type="filepath", label="📷 Upload Injury Photo (optional)")
        file = gr.File(label="📄 Upload Medical Report (PDF/DOCX)", file_types=[".pdf", ".docx"])

    map_output = gr.HTML(label="🌍 Nearest Trusted Medical Facility")
    submit = gr.Button("Send")

    submit.click(
        respond,
        inputs=[txt, img, state, file, full_name, home_address, outbound_flight, return_flight, onboarded_state],
        outputs=[txt, chatbot, state, map_output, onboarded_state,
                 full_name, home_address, outbound_flight, return_flight, case_id_display]
    )

demo.launch(share=True)



* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://2fb673e7367d9def85.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Prompt example: 

First of all you need to insert your details so that a case reference number it is assigned to you. 

Full name: Liam Thompson; Home Address: Street View, Liverpool L1 0AE; Outbound flight: 2025-04-12: Return flight: 2025-04-25.

A Case number will be given. 

Prompt options: 

Option1: I’ve had a severe fall and injured my leg. I’m in need of immediate medical assistance. Can you help?

Option 2: Hi there, this is Liam Thompson, I have just had a terrible accident while on holidays in Da Nang. My coordinates are 16.046444, 108.227064 and I need immediate emdical attention due to having a fracture left leg. I attach you an image so that you can check the severity of the injury.  I need an answer straitght away because it is very urgent.  

Option 3: I attach you a medical report that I have just been given. Thanks

# SECTION 5.4. Unitest

In [14]:
# 1. Define the functions to be tested (These should already exist in your notebook)
import os
import json
from datetime import datetime
import random
from geopy.distance import distance

case_directory = "/kaggle/working/case_files"
os.makedirs(case_directory, exist_ok=True)

def generate_case_number():
    return f"GB1-{random.randint(100,999)}-{random.randint(100,999)}"

def initialize_case(full_name, home_address, outbound, inbound):
    case_number = generate_case_number()
    case = {
        "case_number": case_number,
        "full_name": full_name,
        "home_address": home_address,
        "outbound_flight": outbound,
        "return_flight": inbound,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "chat_history": []
    }
    with open(f"{case_directory}/{case_number}.json", "w") as f:
        json.dump(case, f, indent=4)
    return case

def load_case(case_number):
    path = f"{case_directory}/{case_number}.json"
    if os.path.exists(path):
        with open(path, "r") as f:
            return json.load(f)
    return None

def get_country_level(country):
    level_1 = ["United States", "Canada", "United Kingdom", "Germany", "France"]
    level_2 = ["Mexico", "Brazil", "Thailand", "Vietnam", "Turkey"]
    level_3 = ["Nepal", "Kenya", "Nigeria", "Pakistan", "Bangladesh"]

    if country in level_1:
        return "Level 1"
    elif country in level_2:
        return "Level 2"
    elif country in level_3:
        return "Level 3"
    return "Unknown"

def extract_coordinates_from_text(text):
    coords = re.findall(r"(-?\d+\.\d+),\s*(-?\d+\.\d+)", text)
    if coords:
        return float(coords[0][0]), float(coords[0][1])
    return None


# 2. Now, define the Unit Test for those functions

import unittest

class TestMedicalAssistantFunctions(unittest.TestCase):

    def setUp(self):
        # Setup test case directory and data
        self.case_directory = "/kaggle/working/case_files"
        os.makedirs(self.case_directory, exist_ok=True)
        
        self.test_case_data = {
            "full_name": "John Doe",
            "home_address": "123 Test St, Test City",
            "outbound": "2025-04-01",
            "inbound": "2025-04-10"
        }
    
    def test_generate_case_number(self):
        """Test case number generation"""
        case_number = generate_case_number()
        self.assertTrue(case_number.startswith("GB1-"))
        self.assertEqual(len(case_number.split("-")), 3)

    def test_initialize_case(self):
        """Test case initialization and file saving"""
        case = initialize_case(self.test_case_data["full_name"], self.test_case_data["home_address"],
                               self.test_case_data["outbound"], self.test_case_data["inbound"])
        
        self.assertIn("case_number", case)
        case_file_path = f"{self.case_directory}/{case['case_number']}.json"
        self.assertTrue(os.path.exists(case_file_path))

        # Load the case to verify its correctness
        loaded_case = load_case(case['case_number'])
        self.assertEqual(loaded_case["full_name"], self.test_case_data["full_name"])

    def test_get_country_level(self):
        """Test country level detection"""
        level_1_country = "United States"
        level_2_country = "Mexico"
        level_3_country = "Nepal"
        unknown_country = "Mars"
        
        self.assertEqual(get_country_level(level_1_country), "Level 1")
        self.assertEqual(get_country_level(level_2_country), "Level 2")
        self.assertEqual(get_country_level(level_3_country), "Level 3")
        self.assertEqual(get_country_level(unknown_country), "Unknown")

    def test_extract_coordinates_from_text(self):
        """Test extraction of coordinates from text"""
        text_with_coords = "The hospital is located at 13.7489, 100.5562."
        coordinates = extract_coordinates_from_text(text_with_coords)
        self.assertEqual(coordinates, (13.7489, 100.5562))

        text_without_coords = "The hospital is located nearby."
        coordinates = extract_coordinates_from_text(text_without_coords)
        self.assertIsNone(coordinates)

    def tearDown(self):
        # Cleanup after tests
        for filename in os.listdir(self.case_directory):
            file_path = os.path.join(self.case_directory, filename)
            os.remove(file_path)
        os.rmdir(self.case_directory)

# 3. Run the tests
if __name__ == "__main__":
    unittest.main(argv=[''], verbosity=2, exit=False)



test_extract_coordinates_from_text (__main__.TestMedicalAssistantFunctions.test_extract_coordinates_from_text)
Test extraction of coordinates from text ... ok
test_generate_case_number (__main__.TestMedicalAssistantFunctions.test_generate_case_number)
Test case number generation ... ok
test_get_country_level (__main__.TestMedicalAssistantFunctions.test_get_country_level)
Test country level detection ... ok
test_initialize_case (__main__.TestMedicalAssistantFunctions.test_initialize_case)
Test case initialization and file saving ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.012s

OK


# SECTION 5.5. Debugging with Logging

In [15]:
import logging

# Set up logging to file
logging.basicConfig(level=logging.DEBUG, filename="app_debug.log", 
                    format="%(asctime)s - %(levelname)s - %(message)s")

def respond(message, image, chat_history, file, name, address, outbound, inbound, onboarded):
    logging.debug(f"Received message: {message}")
    logging.debug(f"Received image: {image}")
    logging.debug(f"Chat history: {chat_history}")
    logging.debug(f"File: {file}")
    logging.debug(f"Name: {name}, Address: {address}, Outbound: {outbound}, Inbound: {inbound}, Onboarded: {onboarded}")

    # Proceed with your existing code logic...
    if not onboarded:
        logging.debug("Onboarding in progress.")
        if not (name and address and outbound and inbound):
            logging.warning("Incomplete onboarding data.")
            return "❗Please complete onboarding.", chat_history, chat_history, "", onboarded, \
                   gr.update(visible=True), gr.update(visible=True), \
                   gr.update(visible=True), gr.update(visible=True), \
                   gr.update(visible=False)

    # More code logic...
    logging.debug("Processing completed.")


# SECTION 5.6: Refactored Code:

In [16]:
import os
import json
import random
from datetime import datetime
from geopy.distance import distance
import re

# 🌍 Country Levels
LEVEL_1 = ["United States", "Canada", "United Kingdom", "Germany", "France", "Japan", "Australia", "New Zealand", "Sweden", "Norway", "Netherlands", "Switzerland", "Italy", "Spain", "South Korea", "Singapore"]
LEVEL_2 = ["Mexico", "Brazil", "Thailand", "Vietnam", "Turkey", "Malaysia", "Costa Rica", "Serbia", "India", "Philippines", "China", "Chile", "South Africa", "Indonesia", "Egypt", "UAE"]
LEVEL_3 = ["Nepal", "Kenya", "Nigeria", "Pakistan", "Bangladesh", "Ethiopia", "Uganda", "Myanmar", "Cameroon", "Burkina Faso", "Zimbabwe", "DR Congo", "Sudan", "Ghana", "Bolivia"]

# 🏥 Trusted Hospitals with coordinates
HOSPITAL_LOCATIONS = {
    # Level 2 countries
    "Bumrungrad International Hospital": (13.7489, 100.5562),
    "Samitivej Hospital": (13.7300, 100.5684),
    # Add more hospitals as needed
}

CASE_DIRECTORY = "/kaggle/working/case_files"
os.makedirs(CASE_DIRECTORY, exist_ok=True)

def generate_case_number():
    """Generates a unique case number."""
    return f"GB1-{random.randint(100, 999)}-{random.randint(100, 999)}"

def save_case(case_data):
    """Saves case data to a JSON file."""
    file_path = os.path.join(CASE_DIRECTORY, f"{case_data['case_number']}.json")
    with open(file_path, "w") as f:
        json.dump(case_data, f, indent=4)

def load_case(case_number):
    """Loads case data from a JSON file."""
    file_path = os.path.join(CASE_DIRECTORY, f"{case_number}.json")
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return json.load(f)
    return None

def initialize_case(full_name, home_address, outbound, inbound):
    """Initializes a new case and saves it to the file."""
    case_number = generate_case_number()
    case_data = {
        "case_number": case_number,
        "full_name": full_name,
        "home_address": home_address,
        "outbound_flight": outbound,
        "return_flight": inbound,
        "created_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "chat_history": []
    }
    save_case(case_data)
    return case_data

def get_country_level(country):
    """Returns the level of the country based on predefined lists."""
    if country in LEVEL_1:
        return "Level 1"
    elif country in LEVEL_2:
        return "Level 2"
    elif country in LEVEL_3:
        return "Level 3"
    return "Unknown"

def extract_coordinates_from_text(text):
    """Extracts coordinates from a string of text."""
    coords = re.findall(r"(-?\d+\.\d+),\s*(-?\d+\.\d+)", text)
    if coords:
        return float(coords[0][0]), float(coords[0][1])
    return None

def find_closest_hospital(user_coords):
    """Finds the closest hospital based on user coordinates."""
    min_dist = float("inf")
    closest = None
    for name, coords in HOSPITAL_LOCATIONS.items():
        dist_km = distance(user_coords, coords).km
        if dist_km < min_dist:
            min_dist = dist_km
            closest = (name, coords, dist_km)
    return closest

def generate_map(user_text):
    """Generates an HTML map showing the user's location and the nearest hospital."""
    coords = extract_coordinates_from_text(user_text)
    if not coords:
        return "<p>No coordinates detected in message.</p>"
    closest = find_closest_hospital(coords)
    if not closest:
        return "<p>No hospital found nearby.</p>"
    hname, hcoords, dist = closest
    fmap = folium.Map(location=coords, zoom_start=10)
    folium.Marker(location=coords, popup="Client Location", icon=folium.Icon(color="blue")).add_to(fmap)
    folium.Marker(location=hcoords, popup=f"{hname} ({dist:.1f} km)", icon=folium.Icon(color="green")).add_to(fmap)
    tmp_file = tempfile.NamedTemporaryFile(suffix=".html", delete=False)
    fmap.save(tmp_file.name)
    with open(tmp_file.name, "r") as f:
        return f.read()

def medical_chat(user_input, image=None, chat_history=[]):
    """Simulates medical chat interaction, generating appropriate responses."""
    system_prompt = {
        "role": "system",
        "content": (
            "You are an experienced agent working in the Operations or Medical Team "
            "of a travel health insurance company. Respond empathetically and professionally."
        )
    }

    lower_input = user_input.lower()
    country_found = next((c for c in LEVEL_1 + LEVEL_2 + LEVEL_3 if c.lower() in lower_input), None)
    hospital_found = next((h for h in HOSPITAL_LOCATIONS if h.lower() in lower_input), None)

    guidance = ""
    coords = extract_coordinates_from_text(user_input)
    if coords:
        closest = find_closest_hospital(coords)
        if closest:
            hname, hcoords, dist_km = closest
            dist_mi = dist_km * 0.621371
            transport = "an ambulance" if "severe" in lower_input or "bleeding" in lower_input or "can’t walk" in lower_input else "a taxi"
            maps_link = f"https://www.google.com/maps/dir/{coords[0]},{coords[1]}/{hcoords[0]},{hcoords[1]}"
            guidance = (
                f"🚑 Given your injury, it's crucial to seek care quickly. I recommend **{hname}**, "
                f"which is approximately **{dist_mi:.1f} miles** from your current location.\n\n"
                f"Please arrange for **{transport}** to take you there.\n\n"
                f"📍 [Click here for directions on Google Maps]({maps_link})\n\n"
                "Once you arrive, please confirm admission so we can begin coordinating follow-up care or repatriation if necessary."
            )
            chat_history.append({"role": "assistant", "content": guidance})

    elif country_found:
        level = get_country_level(country_found)
        if level == "Level 1":
            guidance = f"🟢 {country_found} is a Level 1 country. All hospitals are considered reliable."
        elif hospital_found:
            guidance = f"🟢 {hospital_found} in {country_found} is a trusted facility in our network. Care should be appropriate."
        else:
            guidance = (
                f"⚠️ {country_found} is a {level} country. If the hospital is not in our trusted network, "
                "we recommend moving the patient to a reliable facility or considering evacuation."
            )
        chat_history.append({"role": "assistant", "content": guidance})

    if image:
        image_caption = generate_image_description(image)
        chat_history.append({"role": "assistant", "content": f"🖼️ Injury analysis: {image_caption}"})
        if "deep" in image_caption or "open wound" in image_caption or "fracture" in image_caption:
            chat_history.append({"role": "assistant", "content": "⚠️ This injury appears serious. Immediate evaluation is required."})

    prompt_text = f"{user_input}\nImage description: {image_caption}" if image else user_input
    messages = [system_prompt] + chat_history + [{"role": "user", "content": prompt_text}]
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=messages,
        max_tokens=700
    )

    reply = response.choices[0].message.content
    chat_history.append({"role": "user", "content": user_input})
    chat_history.append({"role": "assistant", "content": reply})
    return reply, chat_history


# -----------------------------------
# 📚 SECTION 6: References
# -----------------------------------

In [17]:
references = [
    "📄 Prompt Engineering Whitepaper – Google Cloud",
    "📄 Solving Domain-Specific Problems using LLMs – Google Cloud",
    "📄 Operationalizing Generative AI on Vertex AI – Google Cloud",
    "📄 Agents Whitepaper – Google Cloud",
    "📄 Agents Companion Guide – Vertex AI",
    "📚 LangChain & LangGraph Documentation – https://docs.langchain.com/",
    "🏁 CrewAI Multi-Agent Framework – https://docs.crewai.io/",
    "🏆 Kaggle Competition: 5-Day Generative AI Intensive by Google & DeepMind (2025)",
    "🧠 L1–L6 Notebooks from Google’s Gen AI Capstone on Kaggle",
    "📝 Internal Medical Protocols & ACC Guidelines (uploaded images)",
    "💻 Gemini Model API – via Google Vertex AI",
    "🧪 Streamlit + Gradio for Agent Simulation UI",
    "🔒 GDPR Guidelines – EU Data Protection Regulation",
    "📦 Full project source: https://github.com/OYanez85/5-Day_Gen_AI_Intensive_by_Kaggle_and_Goggle_Deep_Mind_2025",
    "📂 CSV Logs stored in: /kaggle/working/conversation_log.csv"
]

print("\n--- 📚 References ---")
for ref in references:
    print(f"- {ref}")




--- 📚 References ---
- 📄 Prompt Engineering Whitepaper – Google Cloud
- 📄 Solving Domain-Specific Problems using LLMs – Google Cloud
- 📄 Operationalizing Generative AI on Vertex AI – Google Cloud
- 📄 Agents Whitepaper – Google Cloud
- 📄 Agents Companion Guide – Vertex AI
- 📚 LangChain & LangGraph Documentation – https://docs.langchain.com/
- 🏁 CrewAI Multi-Agent Framework – https://docs.crewai.io/
- 🏆 Kaggle Competition: 5-Day Generative AI Intensive by Google & DeepMind (2025)
- 🧠 L1–L6 Notebooks from Google’s Gen AI Capstone on Kaggle
- 📝 Internal Medical Protocols & ACC Guidelines (uploaded images)
- 💻 Gemini Model API – via Google Vertex AI
- 🧪 Streamlit + Gradio for Agent Simulation UI
- 🔒 GDPR Guidelines – EU Data Protection Regulation
- 📦 Full project source: https://github.com/OYanez85/5-Day_Gen_AI_Intensive_by_Kaggle_and_Goggle_Deep_Mind_2025
- 📂 CSV Logs stored in: /kaggle/working/conversation_log.csv
